# Imports and function definitions

In [2]:
import sys
sys.path.append(
    '..'
)  ## <-- THIS IS THE PART THAT TELLS IT TO LOOK IN THE PARENT DIRECTORY
from IPM_Shared_Code.Python.geo_functions import read_geosql

In [3]:
import pandas as pd
import geopandas as gpd
import numpy as np
import pyodbc
import sys
import json
import urllib
import sqlalchemy
import os
import shapely
import datetime
import math
# from datetime import datetime

from arcgis.gis import GIS
from arcgis.features import GeoAccessor, GeoSeriesAccessor, SpatialDataFrame, FeatureLayer, FeatureSet
from geojson import Point, Feature, FeatureCollection, dump
sys.path.append('../')
from IPM_Shared_Code_public.Python.geo_functions import read_geosql
from IPM_Shared_Code_public.Python.utils import get_config
# from IPM_Shared_Code.Python.email_functions import get_contacts, read_template, send_email
# from geosupport import *

C:\Users\Julie.Tsitron\AppData\Local\Continuum\anaconda3\lib\site-packages\arcgis\features\_data\geodataset\utils.py:15: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.datetime,


In [4]:
def get_int(row,col):
    if  math.isnan(row[col]):
        return pd.NA
    else:
        return int(row[col])

In [5]:
def get_date(row,col):
    try:
        return datetime.datetime.fromtimestamp(int(row[col]*10**-3))
    except:
        print(row[col], 'type:', type(row[col]))

In [6]:
def to_wkt(row):
    return row.wkt

In [7]:
config = get_config('C:\Projects\config.ini')

driver = config['srv']['driver']
data_parks_server = config['srv']['data_parks']
server = config['srv']['server']
AGOL_username = config['agol']['user']
AGOL_password = config['agol']['password']

In [8]:
con_string = 'Driver={SQL Server};Server='+server+';Database=dwh;Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [9]:
gis = GIS("https://www.arcgis.com", username=AGOL_username, password = AGOL_password)

In [10]:
con_vpipm = pyodbc.connect(
    'Driver={SQL Server};Server='+server+';Database=DWH;Trusted_Connection=Yes;'
)

In [11]:
con_data_parks = pyodbc.connect(
    'Driver={SQL Server};Server='+data_parks_server+';Database=DWH;Trusted_Connection=Yes;'
)

In [12]:
today = datetime.datetime.today()

In [13]:
sql_AF_active = '''
select * from [DWH].[dbo].[TBL_PARKSGIS_ATHLETICFACILITY]
'''

In [14]:
sql_AF_non_active = '''
select * from [DWH].[dbo].[TBL_NONACTIVE_PARKSGIS_ATHLETICFACILITY]
'''

In [15]:
AF_active_parkgis = read_geosql(sql_AF_active, con_data_parks, geom_raw = 'Shape', geom_col = 'geometry')

C:\Users\Julie.Tsitron\AppData\Local\Continuum\anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


# AGOL layers

In [16]:
pgs = 'b7f5001abc354a5483da9e11bea165f6'

In [17]:
comfort_stations = 'c0d9e93afc234ee1ad82ae87deda8df3'

In [18]:
adult_equip = 'e68e4b82579d4054bba92331bfa0928d'

In [19]:
dog_runs = 'bc44c0d26bca49af8409cb57ff9201ce'

In [20]:
athletic_facilities = '6a878882793b43a187bf4b0d84e23fc8'

In [21]:
skate_parks = '9b2b8cc37cf04fcda5dc7347c4fc759f'

## Start here

In [22]:
sp_item = gis.content.get(skate_parks)
sp_features = sp_item.layers[0].query()

In [23]:
dr_item = gis.content.get(dog_runs)
dr_features = dr_item.layers[0].query()

In [24]:
ae_item = gis.content.get(adult_equip)
ae_features = ae_item.layers[0].query()

In [25]:
af_item = gis.content.get(athletic_facilities)
af_features = af_item.layers[0].query()

In [26]:
pg_item = gis.content.get(pgs)
pg_features = pg_item.layers[0].query()

In [27]:
cs_item = gis.content.get(comfort_stations)
cs_features = cs_item.layers[0].query()

## date columns because they need to be fixed in some cases

In [28]:
pg_date_cols = ['statusChangeDate', 'CreationDate', 'EditDate']

In [29]:
dr_date_cols = ['DateDecommissioned', 'CreationDate', 'EditDate']

In [30]:
ae_date_cols = ['CreationDate', 'EditDate']

In [31]:
af_date_cols = [
    'FeatureStatusChangeDate', 'CreationDate', 'EditDate', 'DateDecommissioned'
]

In [32]:
sp_date_cols = [
    'CreatedDate', 'LastEditedDate', 'DateDecommissioned', 'CreationDate',
    'EditDate'
]

In [33]:
cs_date_cols = ['CommissionDate', 'ConstructionYear', 'AlterationYear', 'DemolitionYear', 'CreationDate',  'EditDate']

## convert feature layers to geojson so they can be converted to geopandas dataframes

In [34]:
layers = ['pg','ae','dr','sp','af','cs']

In [35]:
feature_collection_pg = json.loads(pg_features.to_geojson)
feature_collection_ae = json.loads(ae_features.to_geojson)
feature_collection_dr = json.loads(dr_features.to_geojson)
feature_collection_sp = json.loads(sp_features.to_geojson)
feature_collection_af = json.loads(af_features.to_geojson)
feature_collection_cs = json.loads(cs_features.to_geojson)

feat_collections = [
    feature_collection_pg, feature_collection_ae, feature_collection_dr,
    feature_collection_sp, feature_collection_af, feature_collection_cs
]

# with open('myfile.geojson_af_0513', 'w') as f:
#     dump(feature_collection_af, f)

In [36]:
for i in zip(layers,feat_collections):
    with open(i[0]+'_'+str(today.date())+'.geojson', 'w') as f:
        dump(i[1], f)

## read geojson files into geodataframes

In [ ]:
pg_gdf = gpd.read_file('pg'+'_'+str(today.date())+'.geojson')

In [38]:
ae_gdf = gpd.read_file('ae'+'_'+str(today.date())+'.geojson')

In [39]:
dr_gdf = gpd.read_file('dr'+'_'+str(today.date())+'.geojson')

In [40]:
sp_gdf = gpd.read_file('sp'+'_'+str(today.date())+'.geojson')

In [41]:
af_gdf = gpd.read_file('af'+'_'+str(today.date())+'.geojson')

In [42]:
cs_gdf = gpd.read_file('cs'+'_'+str(today.date())+'.geojson')

### fix dates

In [43]:
date_cols = [
    pg_date_cols, ae_date_cols, dr_date_cols, sp_date_cols, af_date_cols,
    cs_date_cols
]

In [44]:
gdfs = [pg_gdf, ae_gdf, dr_gdf, sp_gdf, af_gdf, cs_gdf]

In [45]:
for i in pg_date_cols:
    pg_gdf[i] = pg_gdf.apply(lambda row: get_int(row, i),axis=1)

In [46]:
for i in ae_date_cols:
    ae_gdf[i] = ae_gdf.apply(lambda row: get_int(row, i),axis=1)

In [47]:
for i in dr_date_cols:
    dr_gdf[i] = dr_gdf.apply(lambda row: get_int(row, i),axis=1)

In [48]:
for i in sp_date_cols:
    sp_gdf[i] = sp_gdf.apply(lambda row: get_int(row, i),axis=1)

In [49]:
for i in af_date_cols:
    af_gdf[i] = af_gdf.apply(lambda row: get_int(row, i),axis=1)

In [50]:
# for i in cs_date_cols:
#     cs_gdf[i] = cs_gdf.apply(lambda row: get_int(row, i),axis=1)

In [51]:
# cs_gdf[cs_date_cols]

In [52]:
# cs_gdf.AlterationYear.values

In [90]:
# def get_date2(row,col):
#     try:
#         return datetime.datetime(1970, 1, 1) + datetime.timedelta(seconds=(-int(row[col])/1000))
#     except:
#         print(row[col])


In [54]:
for i in pg_date_cols:
    pg_gdf[i] = pg_gdf.apply(lambda row: get_date(row, i),axis=1)

<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'p

<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'p

In [55]:
for i in ae_date_cols:
    ae_gdf[i] = ae_gdf.apply(lambda row: get_date(row, i),axis=1)

In [56]:
ae_gdf.head()

,OBJECTID,SYSTEM,PropNum,PropID,SiteName,PropName,Feature,FeatureType,covidStatus,ClosureType,ClosureNotes,GlobalID,CreationDate,Creator,EditDate,Editor,Borough,geometry
0,1,102.0,B007,B007,Bensonhurst Park,Bensonhurst Park,Play Equipment,Exercise Equipment,COVID-19 Closure,Completely Locked,insufficient social distancing observed,abeb238c-e72c-4258-a92c-48713f70b511,2020-04-08 08:22:09,bblackshear,2020-04-10 15:59:47,Jasmine.Soltani_BK,B,POINT (983888.347 156602.127)
1,2,115.0,B007,B007-01,Bensonhurst Park Playground,Bensonhurst Park,Play Equipment,Exercise Equipment,COVID-19 Closure,Completely Locked,insufficient social distancing observed,308d052c-177c-4ea5-b2ca-cee05c0f7a7b,2020-04-08 08:22:09,bblackshear,2020-04-10 16:00:19,Jasmine.Soltani_BK,B,POINT (984301.851 156751.751)
2,3,154.0,B008,B008-03,Imagination Playground at Betsy Head,Betsy Head Park,Play Equipment,Exercise Equipment,COVID-19 Closure,Blocked with Barricade,None,b471d0da-383d-4dfb-838f-d7c8b4b3b1f4,2020-04-08 08:22:09,bblackshear,2020-04-10 17:01:06,Jasmine.Soltani_BK,B,POINT (1008711.433 181388.252)
3,4,227.0,B016,B016,Maria Hernandez Park,Maria Hernandez Park,Play Equipment,Exercise Equipment,Active,None,None,59121fff-0e21-4816-a549-fada95aa6d19,2020-04-08 08:22:09,bblackshear,2020-04-09 15:15:34,Annarita.Macri,B,POINT (1005360.212 195469.964)
4,5,50247.0,B018,B018-01,Dinapoli Plgd,Canarsie Park,Play Equipment,Exercise Equipment,Active,None,None,98de7305-c3d4-4680-bcb2-8019540bbc70,2020-04-08 08:22:09,bblackshear,2020-04-09 15:15:34,Annarita.Macri,B,POINT (1013952.343 169121.315)


In [57]:
for i in dr_date_cols:
    dr_gdf[i] = dr_gdf.apply(lambda row: get_date(row, i),axis=1)

<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'p

In [58]:
for i in sp_date_cols:
    sp_gdf[i] = sp_gdf.apply(lambda row: get_date(row, i),axis=1)

<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'p

In [59]:
for i in af_date_cols:
    af_gdf[i] = af_gdf.apply(lambda row: get_date(row, i),axis=1)

<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'p

<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'p

<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'p

<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'p

<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'pandas._libs.missing.NAType'>
<NA> type: <class 'p

In [60]:
for i in cs_date_cols:
    cs_gdf[i] = cs_gdf.apply(lambda row: get_date(row, i),axis=1)

-619200000000.0 type: <class 'float'>
-419616000000.0 type: <class 'float'>
-482342400000.0 type: <class 'float'>
-974037600000.0 type: <class 'float'>
-269366400000.0 type: <class 'float'>
-578592000000.0 type: <class 'float'>
-590256000000.0 type: <class 'float'>
-175017600000.0 type: <class 'float'>
-2209143600000.0 type: <class 'float'>
-2209143600000.0 type: <class 'float'>
-2209143600000.0 type: <class 'float'>
-2177607600000.0 type: <class 'float'>
-2209143600000.0 type: <class 'float'>
-2177607600000.0 type: <class 'float'>
-2177607600000.0 type: <class 'float'>
-2209143600000.0 type: <class 'float'>
-2209143600000.0 type: <class 'float'>
-2209143600000.0 type: <class 'float'>
-2209143600000.0 type: <class 'float'>
-420998400000.0 type: <class 'float'>
-2177607600000.0 type: <class 'float'>
-2209143600000.0 type: <class 'float'>
-2177607600000.0 type: <class 'float'>
-455299200000.0 type: <class 'float'>
-2209143600000.0 type: <class 'float'>
-2209143600000.0 type: <class 'floa

nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <c

nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <class 'float'>
nan type: <c

None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type: <class 'NoneType'>
None type:

# prepare to write to sql

In [91]:
pg_gdf['geometry'] = pg_gdf['geometry'].apply(to_wkt)
ae_gdf['geometry'] = ae_gdf['geometry'].apply(to_wkt)
dr_gdf['geometry'] = dr_gdf['geometry'].apply(to_wkt)
sp_gdf['geometry'] = sp_gdf['geometry'].apply(to_wkt)
af_gdf['geometry'] = af_gdf['geometry'].apply(to_wkt)

In [62]:
cs_gdf['geometry'] = cs_gdf['geometry'].apply(to_wkt)

In [106]:
# AF = af_gdf.merge(AF_active_parkgis[['System', 'geometry'
#                                      ]].rename(columns={'geometry': 'shape'}),
#                   how='left',
#                   on='System')

In [107]:
# AF

In [108]:
# af_gdf2 = AF[~pd.isnull(AF['shape'])].copy()

In [109]:
# af_gdf2['geometry'] = af_gdf2['geometry'].apply(to_wkt)

In [110]:
# af_gdf2.geometry.name

In [111]:
# af_gdf2[pd.isnull(af_gdf2['geometry'])]

In [66]:
# af_gdf2 = af_gdf2.set_geometry('shape')

In [67]:
# af_gdf2['shape'] = np.asarray(af_gdf2['shape'])

In [68]:
# af_gdf2.geometry.name

In [69]:
# af_gdf2['shape']

In [93]:
# af_gdf['geometry'] = af_gdf['geometry'].apply(to_wkt)

In [102]:
# af_gdf[af_gdf['geometry'].isna()]
# 

In [73]:
# af_gdf2.drop('shape',axis=1)

In [74]:
# af_gdf2['shape']

# Write to SQL

In [75]:
pg_gdf.to_sql('tbl_covid_closures_playgrounds', engine, if_exists='replace')

C:\Users\Julie.Tsitron\AppData\Local\Continuum\anaconda3\lib\site-packages\sqlalchemy\dialects\mssql\base.py:2678: SAWarning: Did not recognize type 'geometry' of column 'geometry'
  % (type_, name)


In [76]:
cursor = con_vpipm.cursor()
cursor.execute("""
    alter table [dwh].[dbo].[tbl_covid_closures_playgrounds] 
    alter column geometry geometry
    """)
con_vpipm.commit()
cursor.close()

In [77]:
ae_gdf.to_sql('tbl_covid_closures_adultequipment', engine, if_exists='replace')

In [78]:
cursor = con_vpipm.cursor()
cursor.execute("""
    alter table [dwh].[dbo].[tbl_covid_closures_adultequipment] 
    alter column geometry geometry
    """)
con_vpipm.commit()
cursor.close()

In [79]:
dr_gdf.to_sql('tbl_covid_closures_dogruns', engine, if_exists='replace')

In [80]:
cursor = con_vpipm.cursor()
cursor.execute("""
    alter table [dwh].[dbo].[tbl_covid_closures_dogruns] 
    alter column geometry geometry
    """)
con_vpipm.commit()
cursor.close()

In [81]:
sp_gdf.to_sql('tbl_covid_closures_skateparks', engine, if_exists='replace')

In [82]:
cursor = con_vpipm.cursor()
cursor.execute("""
    alter table [dwh].[dbo].[tbl_covid_closures_skateparks] 
    alter column geometry geometry
    """)
con_vpipm.commit()
cursor.close()

In [83]:
# af_gdf2[pd.isnull(af_gdf2['shape'])]

In [84]:
# TypeError: Cannot convert GeometryArray to numpy.ndarray

In [103]:
af_gdf.to_sql('tbl_covid_closures_athleticfacilities', engine, if_exists='replace')

C:\Users\Julie.Tsitron\AppData\Local\Continuum\anaconda3\lib\site-packages\sqlalchemy\dialects\mssql\base.py:2678: SAWarning: Did not recognize type 'geometry' of column 'shape'
  % (type_, name)


In [112]:
# af_gdf2.drop('shape',axis=1).to_sql('tbl_covid_closures_athleticfacilities', engine, if_exists='replace')

In [104]:
cursor = con_vpipm.cursor()
cursor.execute("""
    alter table [dwh].[dbo].[tbl_covid_closures_athleticfacilities] 
    alter column geometry geometry
    """)
con_vpipm.commit()
cursor.close()

In [105]:
cursor = con_vpipm.cursor()
cursor.execute("""
    USE DWH;  
    EXEC sp_rename '[dwh].[dbo].[tbl_covid_closures_athleticfacilities].geometry', 'shape', 'COLUMN';  
    """)
con_vpipm.commit()
cursor.close()

In [88]:
cs_gdf.to_sql('tbl_covid_closures_comfortstations', engine, if_exists='replace')

In [89]:
cursor = con_vpipm.cursor()
cursor.execute("""
    alter table [dwh].[dbo].[tbl_covid_closures_comfortstations] 
    alter column geometry geometry
    """)
con_vpipm.commit()
cursor.close()